In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [4]:
from xgboost import XGBClassifier, cv, DMatrix

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score, precision_score, recall_score
from google.cloud import datastore, bigquery
from sklearn.model_selection import *
import numpy as np
import pandas as pd
import os, warnings
import tensorflow as tf
import joblib, json
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, RepeatVector, Input, Reshape, Flatten
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from tensorflow.keras.models import Model
from datapreprocessing import *
nltk.download('punkt')  # For tokenization
nltk.download('stopwords')  # Download stop words
nltk.download('wordnet')  # For lemmatization
from datapreprocessing import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
df = pd.read_excel(open('Consolidated.xlsx', 'rb'), sheet_name='Consolidated')
df_final_test = df.query('Foldername == "fold1"')
df = df.query('Foldername != "fold1"')
df['Review'] = df['Review'].apply(preprocess_text)

In [13]:
df['Review']

80      completely blown away hotel magnificent got gr...
81      weve returned two night stay affinia chicago v...
82      excellent location feel like boutique hotel ri...
83      travel lot business quite frankly expectation ...
84      visited 40th birthday never chicago sure stay ...
                              ...                        
3195    overall hotel okay though problem price accomm...
3196    pleased recent stay palmer house hilton room r...
3197    visit started wrong foot entered lobby struggl...
3198    though grand brand hotel seems disappoint cert...
3199    palmer house hilton look good picture outside ...
Name: Review, Length: 2880, dtype: object

In [11]:
vectorizer = TfidfVectorizer(max_features=150)
le = LabelEncoder()
le.fit(df['Target'])
vectorizer.fit(df['Review'])

TfidfVectorizer(max_features=150)

In [20]:
encoded_labels = le.fit_transform(df['Target'])

In [17]:
tfidf_vectors = vectorizer.transform(df['Review']).toarray()

In [18]:
tfidf_vectors

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.17009809, 0.        ,
        0.1881663 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.24669434, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.16557345,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [8]:
tfidf_vectors.shape

(2880, 1024)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_vectors,  le.transform(df['Target']), test_size=0.2, random_state=42)


In [22]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [24]:
from sklearn import svm

In [25]:
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1).fit(X_train, y_train)

In [26]:
poly_pred = poly.predict(X_test)
rbf_pred = rbf.predict(X_test)

In [27]:
poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

Accuracy (Polynomial Kernel):  91.15
F1 (Polynomial Kernel):  91.17


In [28]:
rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

Accuracy (RBF Kernel):  68.75
F1 (RBF Kernel):  68.75


In [23]:
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.76      0.77       143
           1       0.72      0.72      0.72       134
           2       0.68      0.75      0.71       147
           3       0.75      0.70      0.72       152

    accuracy                           0.73       576
   macro avg       0.73      0.73      0.73       576
weighted avg       0.73      0.73      0.73       576

